# Import Library #
- pip install numpy
- pip install matplotlib
- pip install pandas

In [70]:
import numpy as np
import pandas as pd
import pymysql

### Read Data ###

In [71]:
df = pd.read_excel('../Data Potensi CME Malang Juli 2024.xlsx')
df.head()

,id,divisi,abbr_d,wilayah,abbr_w,sto,abbr_s,ruangan,kategori,tipe_perangkat,...,serial_number,kap_tps,kap_tpk,satuan,kondisi,status,periode,qrcode,keterangan,reg_date
0,2615,REGIONAL 5,TR5,MALANG,ML,MALANG KOTA,MLG,R. MITRA,AC SPLIT,AC SPLIT,...,E055728,24000.0,24000.0,BTUH,NORMAL,OPERASI,2008,R503473,SPLIT,2021-06-21 04:33:32
1,2616,REGIONAL 5,TR5,MALANG,ML,MALANG KOTA,MLG,R. BATTERY NEW NGN,AC SPLIT,AC SPLIT,...,37366 7329 INDOOR,18000.0,18000.0,BTUH,NORMAL,OPERASI,2017,R502993,SPLIT,2021-06-21 04:33:32
2,2617,REGIONAL 5,TR5,MALANG,ML,MALANG KOTA,MLG,R. BATTERY NEW NGN,AC SPLIT,AC SPLIT,...,373 663 8247,18000.0,18000.0,BTUH,NORMAL,OPERASI,2017,R503203,SPLIT,2021-06-21 04:33:32
3,2618,REGIONAL 5,TR5,MALANG,ML,MALANG KOTA,MLG,R. DWDM,AC SPLIT,AC SPLIT,...,NaN,24000.0,24000.0,BTUH,NORMAL,OPERASI,2005,R503033,SPLIT,2021-06-21 04:33:32
4,2619,REGIONAL 5,TR5,MALANG,ML,MALANG KOTA,MLG,R.DDF,AC SPLIT,AC SPLIT,...,E0019985,24000.0,24000.0,BTUH,NORMAL,OPERASI,2010,R502733,SPLIT,2021-06-21 04:33:32


# Making Pivot #

In [72]:
category_counts = df.groupby(['sto', 'kategori']).size().reset_index(name='total')

# Pivot the table
pivot_table = category_counts.pivot_table(index='sto', columns='kategori', values='total', fill_value=0)

pivot_table = pivot_table.astype(int)

pivot_table['Total'] = pivot_table.sum(axis=1)
# Reset index to make 'sto' a column again
pivot_table = pivot_table.reset_index()


# pivot_table.head(15)



## Sorting the year ##

#### Converting the Age ####

In [73]:
current_year=2024

df['age'] = current_year - df['periode']


#### Count the Number of Products in Each Age Range ####

In [74]:
bins = [0, 5, 10, df['age'].max()]
labels = ['underfive', 'morethanfive', 'morethanten']

# Create a new column in the DataFrame based on the conditions
df['age_range'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)

# Group by 'sto' and 'age_range' and count occurrences
age_range_counts = df.groupby(['sto', 'age_range']).size().unstack(fill_value=0).reset_index()

print(age_range_counts)

age_range                   sto  underfive  morethanfive  morethanten
0                   AMPELGADING          3             1           26
1                        BANTUR          3             1           18
2                          BATU          4             2           24
3                      BLIMBING          4             4           27
4                        BURING          3             2           24
5                        DAMPIT          1             2           25
6                     DONOMULYO          2             1           20
7                        GADANG          3             3           22
8                  GONDANG LEGI          4             0           23
9                   GUNUNG KAWI          4             3           17
10                 KARANG PLOSO          2             1           24
11                     KEPANJEN          3             0           21
12                       KLOJEN          1             2           30
13                  

C:\Users\ryzen\AppData\Local\Temp\ipykernel_13144\4098047007.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_range_counts = df.groupby(['sto', 'age_range']).size().unstack(fill_value=0).reset_index()


## Combining Tables ##

In [75]:
combined_df = pd.merge(pivot_table, age_range_counts, on='sto')

combined_df.head(40)

,sto,AC PRESISI,AC SPLIT,BATTERE,CAPASITOR BANK,DIESEL ENGINE / AMF,DIESEL MOBILE,GROUNDING,INSTALASI DAYA,INVERTER,OSASE,RECTIFIER,Total,underfive,morethanfive,morethanten
0,AMPELGADING,0,6,3,0,4,0,3,10,1,1,2,30,3,1,26
1,BANTUR,0,4,4,0,4,0,3,3,1,1,2,22,3,1,18
2,BATU,0,9,6,0,4,0,0,3,1,1,6,30,4,2,24
3,BLIMBING,0,6,8,1,6,0,1,4,1,1,7,35,4,4,27
4,BURING,0,6,5,0,4,2,1,7,1,1,2,29,3,2,24
5,DAMPIT,0,7,5,0,6,0,2,3,1,1,3,28,1,2,25
6,DONOMULYO,0,3,4,0,3,1,1,7,1,1,2,23,2,1,20
7,GADANG,0,8,3,0,7,0,1,5,1,1,2,28,3,3,22
8,GONDANG LEGI,0,5,6,0,5,0,1,5,1,1,4,28,4,0,23
9,GUNUNG KAWI,0,3,4,0,4,0,1,8,1,1,2,24,4,3,17


### Importing To MySql ###

In [76]:
db_host = 'localhost'
db_user = 'root'
db_password = ''
db_name = 'arnet'

connection = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)
sto = combined_df['sto'].unique()
sto = sorted(sto)
sto_mapping = {s: index + 1 for index, s in enumerate(sto)}

# Step 2: Apply the mapping to the DataFrame
combined_df['sto_id'] = combined_df['sto'].map(sto_mapping)

try:
    # Create a cursor object
    cursor = connection.cursor()
    
    # Truncate the table to clear existing data
    cursor.execute("TRUNCATE TABLE cmes")

    # Define the SQL INSERT stateme nt
    insert_query = """
    INSERT INTO cmes (sto_id, type_id, underfive, morethanfive, morethanten, total)
    VALUES (%s, %s, %s, %s, %s, %s)
    """

    # Iterate over the DataFrame rows and insert each row into the table
    for index, row in combined_df.iterrows():
        cursor.execute(insert_query, (
            row['sto_id'], 
            1,  # Assuming type_id is always 1
            row['underfive'],
            row['morethanfive'],
            row['morethanten'],
            row['Total']
        ))
        
    # Commit the transaction
    connection.commit()

finally:
    # Close the cursor and the connection
    cursor.close()
    connection.close()

print("Data inserted successfully")


Data inserted successfully


# This is for creating seeder only run if neccesery #

In [78]:
# Get unique values of 'sto', 'ruangan', and 'kategori' columns
sto = df['sto'].unique()
sto = sorted(sto)
rooms = df['ruangan'].unique()
category = df['kategori'].unique()

# Prepare the data for seeder
dropdowns = {}

# Create a dictionary with 'sto' values
for index, s in enumerate(sto):
    dropdowns[str(index)] = { 
        'type': ['sto'],
        'subtype': [s]
    }

# Create a dictionary with 'room' values starting from len(sto)
start_index = len(sto)
for index, room in enumerate(rooms):
    dropdowns[str(start_index + index)] = {  
        'type': ['room'],
        'subtype': [room]
    }

# Create a dictionary with 'category' values starting from len(sto) + len(rooms)
start_index = len(sto) + len(rooms)
for index, c in enumerate(category):
    dropdowns[str(start_index + index)] = {  
        'type': ['category'],
        'subtype': [c]
    }

# Convert the dictionary to PHP-like array syntax
def dict_to_php_array(d):
    php_array = "{\n"
    for key, value in d.items():
        php_array += f"    '{key}' => [\n"
        php_array += f"        'type' => {value['type']},\n"
        php_array += f"        'subtype' => {value['subtype']},\n"
        php_array += "    ],\n"
    php_array += "}\n"
    return php_array

php_array_str = dict_to_php_array(dropdowns)
print(php_array_str)


{
    '0' => [
        'type' => ['sto'],
        'subtype' => ['AMPELGADING'],
    ],
    '1' => [
        'type' => ['sto'],
        'subtype' => ['BANTUR'],
    ],
    '2' => [
        'type' => ['sto'],
        'subtype' => ['BATU'],
    ],
    '3' => [
        'type' => ['sto'],
        'subtype' => ['BLIMBING'],
    ],
    '4' => [
        'type' => ['sto'],
        'subtype' => ['BURING'],
    ],
    '5' => [
        'type' => ['sto'],
        'subtype' => ['DAMPIT'],
    ],
    '6' => [
        'type' => ['sto'],
        'subtype' => ['DONOMULYO'],
    ],
    '7' => [
        'type' => ['sto'],
        'subtype' => ['GADANG'],
    ],
    '8' => [
        'type' => ['sto'],
        'subtype' => ['GONDANG LEGI'],
    ],
    '9' => [
        'type' => ['sto'],
        'subtype' => ['GUNUNG KAWI'],
    ],
    '10' => [
        'type' => ['sto'],
        'subtype' => ['KARANG PLOSO'],
    ],
    '11' => [
        'type' => ['sto'],
        'subtype' => ['KEPANJEN'],
    ],
    '12' 